In [ ]:
import rustworkx as rx
from rustworkx.visualization import mpl_draw as draw_graph
import numpy as np

# FIRST EXAMPLE IN PAPER - G = 10-cycle, H = K_2 (max-cut)
# m = 10
# g = rx.PyGraph()
# g.add_nodes_from(np.arange(0, m, 1))
# e_g_list = [(0, 1, 1.0), (1, 2, 1.0), (2, 3, 1.0), (4, 5, 1.0), (5, 6, 1.0), (6, 7, 1.0), (7, 8, 1.0), (8, 9, 1.0), (9, 0, 1.0), (3, 6, 1.0), (4, 7, 1.0), (2, 8, 1.0), (0, 7, 1.0), (6, 8, 1.0)]
# g.add_edges_from(e_g_list)
# draw_graph(g, node_size=600, with_labels=True, node_color='orange')

# n = 2
# h = rx.PyGraph()
# h.add_nodes_from(np.arange(0, n, 1))
# e_h_list = [(0, 1, 1.0)]
# h.add_edges_from(e_h_list)
# #draw_graph(h, node_size=600, with_labels=True)

# SECOND EXAMPLE IN PAPER - G = 6-cycle, H = K_3 (max-3-cut)
m = 6
g = rx.PyGraph()
g.add_nodes_from(np.arange(0, m, 1))
e_g_list = [(0, 1, 1.0), (1, 2, 1.0), (2, 3, 1.0), (3, 4, 1.0), (4, 5, 1.0), (5, 0, 1.0)]
g.add_edges_from(e_g_list)
draw_graph(g, node_size=600, with_labels=True, node_color='orange')

n = 3
h = rx.PyGraph()
h.add_nodes_from(np.arange(0, n, 1))
e_h_list = [(0, 1, 1.0), (1, 2, 1.0), (0, 2, 1.0)]
h.add_edges_from(e_h_list)
draw_graph(h, node_size=600, with_labels=True)

In [ ]:
e_g = np.zeros((m,m))
for edge in e_g_list:
    e_g[edge[0], edge[1]] = edge[2]
    e_g[edge[1], edge[0]] = edge[2]

e_h = np.zeros((n,n))
for edge in e_h_list:
    e_h[edge[0], edge[1]] = edge[2]
    e_h[edge[1], edge[0]] = edge[2]

print(e_g)
print(e_h)

In [ ]:
def degree(graph, vertex):
    return np.sum(graph[vertex])

# Constructing Q
Q = np.zeros((m*n, m*n))
for i in range(m):
    for j in range(m):
        for u in range(n):
            for v in range(n):
                Q[i*n+u, j*n+v] = e_g[i, j] * e_h[u, v]

# Adding penalty for attempting to map input to multiple outputs
for i in range(m):
    start = i*n
    end = (i+1)*n
    for j in range(start, end):
        for k in range(start, end):
            if j != k:
                Q[j, k] -= degree(e_g, i) * degree(e_h, j - start) # penalty outweighs the highest possible reward for assigning an input to two outputs
print(Q)

In [ ]:
from qiskit.quantum_info import SparsePauliOp

# see https://learning.quantum.ibm.com/tutorial/quantum-approximate-optimization-algorithm

pauli_list = []
for a in range(len(Q)):
    c = 0
    for b in range(len(Q[a])):
        if Q[a, b] != 0:
            paulis = ["I"] * len(Q)
            paulis[a], paulis[b] = "Z", "Z"
            weight = Q[a, b] # should always be 1 for now
            weight *= -1 # -weight to convert from minimizing problem to maximizing
            pauli_list.append(("".join(paulis), weight))

            c -= Q[a, b] + Q[b, a]
    paulis = ["I"] * len(Q)
    paulis[a] = "Z"
    pauli_list.append(("".join(paulis), -c)) # -c to convert from minimizing problem to maximizing
    
print(pauli_list)
cost_hamiltonian = SparsePauliOp.from_list(pauli_list)
print("Cost Function Hamiltonian:", cost_hamiltonian)

In [ ]:
from qiskit.circuit.library import QAOAAnsatz

reps = 10
circuit = QAOAAnsatz(cost_operator=cost_hamiltonian, reps=reps)
circuit.measure_all()

circuit.draw('mpl')

Below cell is not used unless running on IBM hardware

In [ ]:
# from qiskit_ibm_runtime import QiskitRuntimeService
# from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# # QiskitRuntimeService.save_account(channel="ibm_quantum", token="<MY_IBM_QUANTUM_TOKEN>", overwrite=True, set_as_default=True)
# service = QiskitRuntimeService(channel='ibm_quantum')
# backend = service.least_busy(min_num_qubits=127)
# print(backend)

# # Create pass manager for transpilation
# pm = generate_preset_pass_manager(optimization_level=3,
#                                     backend=backend)

# candidate_circuit = pm.run(circuit)
# candidate_circuit.draw('mpl', fold=False, idle_wires=False)

In [ ]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator

backend = AerSimulator()
print(backend)

# Create pass manager for transpilation
pm = generate_preset_pass_manager(optimization_level=3,
                                    backend=backend)

candidate_circuit = pm.run(circuit)

In [ ]:
initial_gamma = np.pi
initial_beta = np.pi/2
init_params = [initial_gamma, initial_beta] * reps

In [ ]:
def cost_func_estimator(params, ansatz, hamiltonian, estimator):

    # transform the observable defined on virtual qubits to
    # an observable defined on all physical qubits
    isa_hamiltonian = hamiltonian.apply_layout(ansatz.layout)

    pub = (ansatz, isa_hamiltonian, params)
    job = estimator.run([pub])

    results = job.result()[0]
    cost = results.data.evs

    objective_func_vals.append(cost)
    # print(cost) - for testing, print cost as optimizer runs

    return cost

In [ ]:
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from scipy.optimize import minimize
import matplotlib.pyplot as plt

objective_func_vals = [] # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)
    # estimator.options.default_shots = 1000

    # Set simple error suppression/mitigation options
    # estimator.options.dynamical_decoupling.enable = True
    # estimator.options.dynamical_decoupling.sequence_type = "XY4"
    # estimator.options.twirling.enable_gates = True
    # estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit, cost_hamiltonian, estimator),
        method="COBYLA",
        tol=1e-3,
    )
    #print(result)

plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

In [ ]:
optimized_circuit = candidate_circuit.assign_parameters(result.x)
#optimized_circuit.draw('mpl', fold=False, idle_wires=False)

In [ ]:
from qiskit_ibm_runtime import SamplerV2 as Sampler

# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Below code is only used for IBM hardware
# Set simple error suppression/mitigation options
# sampler.options.dynamical_decoupling.enable = True
# sampler.options.dynamical_decoupling.sequence_type = "XY4"
# sampler.options.twirling.enable_gates = True
# sampler.options.twirling.num_randomizations = "auto"

pub= (optimized_circuit, )
job = sampler.run([pub], shots=int(1e4))
counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_int = {key: val/shots for key, val in counts_int.items()}
final_distribution_bin = {key: val/shots for key, val in counts_bin.items()}
print(final_distribution_int)

In [ ]:
top_n = 5

def to_bitstring(integer, num_bits):
    result = np.binary_repr(integer, width=num_bits)
    return [int(digit) for digit in result]

keys = list(final_distribution_int.keys())
values = list(final_distribution_int.values())
most_likely = [(keys[i], values[i]) for i in np.argsort(np.abs(values))][:-top_n-1:-1]
most_likely_bitstring = [(to_bitstring(key, len(Q))[::-1], value) for (key, value) in most_likely]

for bit_string in most_likely_bitstring:
    print("Bitstring: ", bit_string)

In [ ]:
most_likely_key_only = [key for (key, value) in most_likely_bitstring]
most_likely_value_only = [value for (key, value) in most_likely_bitstring]

plt.bar(range(len(most_likely_value_only)), most_likely_value_only)

# Add labels
plt.xlabel("Index")
plt.ylabel("Probability")
plt.title("Bar Chart of Phi Probabilities")
plt.show()

hom = np.zeros((top_n, m, n))
for k in range(top_n):
    for i in range(m):
        for j in range(n):
            hom[k][i, j] = most_likely_key_only[k][i*n + j]
    print(f"\nIndex {k}:")
    print(hom[k])

In [ ]:
def first_nonzero(array):
    for i in range(len(array)):
        if array[i] != 0:
            return i
    return None

def edge_in_graph(g, tup):
    if tup == None:
        return 0
    return g[tup[0]][tup[1]]

def edge_across_hom(hom, v1, v2):
    if first_nonzero(hom[v1]) == None or first_nonzero(hom[v2]) == None:
        return None
    return (first_nonzero(hom[v1]), first_nonzero(hom[v2]))

print(f'G:\n{e_g}')
print(f'H:\n{e_h}')

max = 0
max_index = 0
for h in range(len(hom)):
    total = 0
    for i in range(m):
        for j in range(m):
            term = edge_in_graph(e_g, (i,j)) * edge_in_graph(e_h, edge_across_hom(hom[h], i, j))
            total += term
            if term == 1 and h == 1:
                #print(f'Edge in G: ({i}, {j}), edge in H: ({edge_across_hom(hom[h], i, j)})')
                pass
            if i == 2 and j == 3:
                #print(f'(2, 3) Edge in H: ({edge_across_hom(hom[h], i, j)})')
                pass
    #print(f'Value: {total}')
    if total > max:
        max = total
        max_index = h

print(f'Best hom ({max // 2}):\n{hom[max_index]}')